In [ ]:
import os

def get_cropped_faces_paths_and_labels(base_dir):
    cropped_face_paths = []
    labels = []

    # Loop through the 'real_video' and 'fake_video' directories inside 'faces'
    for video_type, label_value in [('real_video', 0), ('fake_video', 1)]:
        video_dir = os.path.join(base_dir, video_type)
        print(f"Looking in directory: {video_dir}")
        if not os.path.exists(video_dir):
            print(f"Directory does not exist: {video_dir}")
            continue


        for subdir in os.listdir(video_dir):
            subdir_path = os.path.join(video_dir, subdir)
            if not os.path.isdir(subdir_path):
                continue


            for face_file in os.listdir(subdir_path):
                if face_file.endswith('.jpg'):
                    cropped_face_paths.append(os.path.join(subdir_path, face_file))

                    labels.append(label_value)

    return cropped_face_paths, labels


base_dir = "/content/drive/MyDrive/crop_face/faces"
cropped_face_paths, labels = get_cropped_faces_paths_and_labels(base_dir)


print(cropped_face_paths)
print(labels)


Looking in directory: /content/drive/MyDrive/crop_face/faces/real_video
Looking in directory: /content/drive/MyDrive/crop_face/faces/fake_video
['/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0000_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0001_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0002_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0003_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0004_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0005_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0006_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0007_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0008_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0009_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import cv2

# Load and preprocess images
def load_images(image_paths, target_size=(128, 128)):
    images = []
    for path in image_paths:
        img = cv2.imread(path)
        img = cv2.resize(img, target_size)
        img = img.astype('float32') / 255.0
        images.append(img)
    return np.array(images)

X = load_images(cropped_face_paths)
y = np.array(labels)

# Split into train and test sets
X_train, X_test, y_val, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.5263 - loss: 0.8837 - val_accuracy: 0.7337 - val_loss: 0.5610
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.7035 - loss: 0.5791 - val_accuracy: 0.7813 - val_loss: 0.4780
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7942 - loss: 0.4430 - val_accuracy: 0.8467 - val_loss: 0.3393
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8642 - loss: 0.3121 - val_accuracy: 0.8814 - val_loss: 0.2562
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8992 - loss: 0.2253 - val_accuracy: 0.9225 - val_loss: 0.1802
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9189 - loss: 0.1824 - val_accuracy: 0.9362 - val_loss: 0.1435
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9439 - loss: 0.1340 - val_accuracy: 0.9362 - val_loss: 0.1428
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9605 - loss: 0.0981 - val_ac

In [ ]:
model.save('model.h5')

In [ ]:
import os

def get_cropped_face_sequences_and_labels(base_dir, sequence_length):
    face_sequences = []
    labels = []

    for video_type, label_value in [('real_video', 0), ('fake_video', 1)]:
        video_dir = os.path.join(base_dir, video_type)
        print(f"Looking in directory: {video_dir}")
        if not os.path.exists(video_dir):
            print(f"Directory does not exist: {video_dir}")
            continue


        for subdir in os.listdir(video_dir):
            subdir_path = os.path.join(video_dir, subdir)
            if not os.path.isdir(subdir_path):
                continue

            face_files = sorted([f for f in os.listdir(subdir_path) if f.endswith('.jpg')])


            for i in range(0, len(face_files) - sequence_length + 1):
                sequence = face_files[i:i + sequence_length]
                face_sequences.append([os.path.join(subdir_path, frame) for frame in sequence])
                labels.append(label_value)

    return face_sequences, labels

base_dir = "/content/drive/MyDrive/crop_face/faces"
sequence_length = 10
face_sequences, labels = get_cropped_face_sequences_and_labels(base_dir, sequence_length)


print(face_sequences)
print(labels)


Looking in directory: /content/drive/MyDrive/crop_face/faces/real_video
Looking in directory: /content/drive/MyDrive/crop_face/faces/fake_video
[['/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0000_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0001_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0002_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0003_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0004_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0005_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0006_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0007_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0008_face_0.jpg', '/content/drive/MyDrive/crop_face/faces/real_video/00295/frame_0009_face_0.jpg'], ['/content/drive/MyDrive/crop_face/faces/re

In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class FrameSequenceGenerator(Sequence):
    def __init__(self, sequences, labels, batch_size, target_size, sequence_length):
        self.sequences = sequences
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.sequence_length = sequence_length

    def __len__(self):
        return int(np.ceil(len(self.sequences) / self.batch_size))

    def __getitem__(self, idx):
        batch_sequences = self.sequences[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        x_batch = np.array([self.load_sequence(sequence) for sequence in batch_sequences])
        y_batch = np.array(batch_labels)

        return x_batch, y_batch

    def load_sequence(self, sequence):

        frames = []
        for frame_path in sequence:
            img = load_img(frame_path, target_size=self.target_size)
            img_array = img_to_array(img) / 255.0
            frames.append(img_array)
        return np.stack(frames)


target_size = (64, 64)
batch_size = 16

train_generator = FrameSequenceGenerator(face_sequences, labels, batch_size, target_size, sequence_length)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential

def build_3d_cnn(input_shape):
    model = Sequential()

    # First 3D Conv layer
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape, padding='same')) # Added padding='same'
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())

    # Second 3D Conv layer
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', padding='same')) # Added padding='same'
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())

    # Third 3D Conv layer
    model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', padding='same')) # Added padding='same'
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(BatchNormalization())

    # Flatten and Fully Connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


input_shape = (sequence_length, 64, 64, 3)
model = build_3d_cnn(input_shape)


model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_6 (Conv3D)                    │ (None, 10, 64, 64, 32)      │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_5 (MaxPooling3D)       │ (None, 5, 32, 32, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 5, 32, 32, 32)       │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_7 (Conv3D)                    │ (None, 5, 32, 32, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_6 (MaxPooling3D)       │ (None, 2, 16, 16, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 2, 16, 16, 64)       │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_8 (Conv3D)                    │ (None, 2, 16, 16, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_7 (MaxPooling3D)       │ (None, 1, 8, 8, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 1, 8, 8, 128)        │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,475,521 (17.07 MB)

 Trainable params: 4,475,073 (17.07 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Fit the model
history = model.fit(train_generator, epochs=10, verbose=1)

# Save the model
model.save('3d_cnn_deepfake_detection.h5')


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


65/65 ━━━━━━━━━━━━━━━━━━━━ 43s 480ms/step - accuracy: 0.5062 - loss: 5.7241
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 30s 413ms/step - accuracy: 0.5897 - loss: 0.8044
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 30s 404ms/step - accuracy: 0.5485 - loss: 0.7434
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 31s 416ms/step - accuracy: 0.5334 - loss: 0.6778
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 40s 400ms/step - accuracy: 0.6282 - loss: 0.7158
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 30s 413ms/step - accuracy: 0.6116 - loss: 0.6436
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 41s 418ms/step - accuracy: 0.6033 - loss: 0.6293
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 30s 421ms/step - accuracy: 0.6160 - loss: 0.5915
Epoch 9/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 41s 414ms/step - accuracy: 0.6614 - loss: 0.7030
Epoch 10/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 30s 407ms/step - accuracy: 0.7360 - loss: 0.5454


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

base_model = EfficientNetB0(include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 101s 166ms/step - accuracy: 0.4926 - loss: 0.7047 - val_accuracy: 0.5295 - val_loss: 0.6926
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.4970 - loss: 0.6935 - val_accuracy: 0.5295 - val_loss: 0.6931
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.5116 - loss: 0.6931 - val_accuracy: 0.5295 - val_loss: 0.6931
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5049 - loss: 0.6931 - val_accuracy: 0.4705 - val_loss: 0.6933
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.4956 - loss: 0.6932 - val_accuracy: 0.4705 - val_loss: 0.6933
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.4908 - loss: 0.6932 - val_accuracy: 0.4705 - val_loss: 0.6933
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.4993 - loss: 0.6932 - val_accuracy: 0.4705 - val_loss: 0.6935
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 

In [ ]:
model.save("detection.h5")

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 144s 452ms/step - accuracy: 0.7230 - loss: 0.5304 - val_accuracy: 0.4705 - val_loss: 0.7035
Epoch 2/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.9421 - loss: 0.1398 - val_accuracy: 0.5295 - val_loss: 0.6934
Epoch 3/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - accuracy: 0.9525 - loss: 0.1082 - val_accuracy: 0.5488 - val_loss: 0.7045
Epoch 4/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - accuracy: 0.9542 - loss: 0.0997 - val_accuracy: 0.7159 - val_loss: 0.5895
Epoch 5/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - accuracy: 0.9725 - loss: 0.0625 - val_accuracy: 0.8450 - val_loss: 0.3184
Epoch 6/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.9668 - loss: 0.0805 - val_accuracy: 0.9322 - val_loss: 0.2042
Epoch 7/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.9724 - loss: 0.0544 - val_accuracy: 0.8757 - val_loss: 0.2890
Epoch 8/10
155/155 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.9713 - loss: 0.0640 - val

In [ ]:
model.save("efficientnet.h5")